# lists all classes in the order of loading

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

class iFile:
    _FILE = None;
    _FILE_PATH=None;
    _FILE_NAME=None;
    _FILE_NAME_DECIPHERS={};
    _FILE_FORMAT=None;
    _FILE_TYPE=None;
    _FOLDERS_DECIPHERS={};
    _FOLDER_PATH=None; #
    _ROOT_FOLDER='../';
    _FULL_PATH = None; #[../][data/hotspot][/x5/25k][models][ker/1][hotspot_ker_1_2020040320300.h5]
    
    _GAME_PATH = None;
    _DATASET_PATH = None    
    _FOLDER_TYPE=None;
    
    
    _INFO = {};

    def __init__(self, FileName, FolderPath='', FileFormat=None):
        self._FILE_NAME = FileName;
        self._FOLDER_PATH = FolderPath;
        self._FILE_FORMAT = FileFormat;
        self.decipher_file_name();
        #self.derive_file_path();
        #self.derive_full_path();
        print("[iFile:__init__]")
        
    def set_root_folder(self, RootFolder):
        self._ROOT_FOLDER = RootFolder;
        print("[iFile:set_root_folder] {0}".format(RootFolder))
        
    def decipher_file_name(self, delimiter="_"):
        print("[iFile:decipher_file_name] Not Implemented");
        
    def derive_game_path(self, DatasetInfo):
        self._GAME_PATH = 'data/' + DatasetInfo['GAME'] + "/";
        
    def derive_dataset_path(self, DatasetInfo):
        self._DATASET_PATH = DatasetInfo['xnINPUTS']  + "/" + DatasetInfo['xnDRAWS'] + "/";
        
        
    def derive_folder_path(self, DatasetInfo):
        self.derive_game_path(DatasetInfo);
        self.derive_dataset_path(DatasetInfo)
        self._FOLDER_PATH = self._GAME_PATH + self._DATASET_PATH;
        print("[iFile:derive_folder_path] {0}".format(self._FOLDER_PATH))
       
   
    def set_file_format(self, FileName):
        #split name and see if file format is included
        Fs=FileName.split(".");
        try:
          self._FILE_FORMAT=Fs[1];
        except IndexError:
          print("File format not included in FIle Name")
        
        print("[iFile:set_file_format] {0}".format(self._FILE_FORMAT))
        return self._FILE_FORMAT;
    
   
    def derive_file_path(self):
        #derive file name from decipher  
        print("[iFile:derive_file_path] Not Implemented")

    def derive_full_path(self):
        FullPath = self._ROOT_FOLDER; #back to /
        FullPath += self._FOLDER_PATH; #derived from id
        FullPath += self._FILE_PATH;
        self._FULL_PATH = FullPath;
        
        print("[ifile:derive_full_path] {0}".format(self._FULL_PATH))
        
    def get_full_path(self):
        print("[iFile:get_full_path] {0}".format(self.__FULL_PATH))
        return self._FULL_PATH;
    
    def ensure_dirs(self):
        path = Path(self._FULL_PATH)
        path.parent.mkdir(parents=True, exist_ok=True)
        
    def getVarType(self, Var):
        varType = ''
        if (type(Var) == pd.core.frame.DataFrame):
            varType = "Dataframe";
        elif (type(Var) == type(None)):
            varType = "Nonetype";
        elif (type(Var) == np.ndarray):
            varType = "NumpyArray";
        #elif(type(df) == np.)
        
        return varType;
        
    def isDataframe(self, Var):
        return self.getVarType(Var) == "Dataframe";
    
    def isNonetype(self, Var):
        return self.getVarType(Var) == "Nonetype";
    
    def isNumpyArray(self, Var):
        return self.getVarType(Var) == "NumpyArray";
    
    def print_file_info(self):
        print("FILE NAME: {0}".format(self._FILE_NAME));
        print("FILE FORMAT(type): {0}[{1}]".format(self._FILE_FORMAT, self._FILE_TYPE));
        print("FILE PATH: {0}".format(self._FILE_PATH));
        print("FILE NAME PARTS: {0}".format(self._INFO));

In [5]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sb
import keras as kr
from numpy import array
#from ipynb.fs.full.ifile import iFile

class Data(iFile):
    _FILE_FORMAT = "pkl"
    _DATA_FOLDER = None;
    _DF_MASTER = None
    _DF_IDs = None;
    _DF_R = None;
    _DF_DR = None;
    _DF_DP = None;
    _NPZ_MASTER = None
    _MIN_ID=0;
    _MAX_ID=0;
    _COUNT_ID=0;
    
    def __init__(self, MasterFileName="hotspot_master", LoadMaster=False, DataFolder="data/"):
        self._FILE_NAME  = MasterFileName;
        self._DATA_FOLDER = DataFolder;
        super().__init__(MasterFileName, DataFolder, self._FILE_FORMAT);
        
        self.derive_folder_path();
        self.derive_file_path();
        self.derive_full_path();
        
        if(LoadMaster == True):
            self.load_master_df();
            self.deconstruct_master_df()
            print("Loaded data file %s" %(self._FULL_PATH))
            
    #v
    def decipher_file_name(self, Delimiter="_"):
        #if .format is included, remove this first
        
        FN_PARTS = self._FILE_NAME.split(Delimiter);
        try:
            self._INFO['GAME'], self._GAME = FN_PARTS[0], FN_PARTS[0]
            self._INFO['FOLDER_TYPE'], self._FOLDER_TYPE = FN_PARTS[1], FN_PARTS[1]
            self._INFO['DATA_TYPE'], self._DATA_TYPE = FN_PARTS[2], FN_PARTS[2]
            #check if file format
            self.set_file_format(self._FILE_NAME)
        except IndexError:
            pass;      
        
    def derive_folder_path(self):
        self._FOLDER_PATH = 'data/' + self._GAME + "/";
        
    def derive_file_path(self):
        self._FILE_PATH = "master" + "/" + self._FILE_NAME +"." + self._FILE_FORMAT;
    
    #v    
    def load_file(self):
        self.set_master_df(pd.read_pickle(self._FULL_PATH));   

            
    def get_data_folder(self):
        return self._DATA_FOLDER;
    
    def load_master_df(self):
        if (self._FILE_NAME != ""):
            self.set_master_df(pd.read_pickle(self._FULL_PATH));
    
    def set_master_df(self, master_df):
        self._DF_MASTER = master_df;
        self._MIN_ID = self._DF_MASTER.min();
        self._MAX_ID = self._DF_MASTER.index.max();
        self._COUNT_ID = self._DF_MASTER.shape[0];
        
    def get_master_df(self):
        return self._DF_MASTER;
    
    def set_master_npz(self, master_npz):
        self._NPZ_MASTER = master_df;
        
    def get_master_npz(self):
        return self._NPZ_MASTER
    
    def get_result_columns(self):
        i=1; r=[];
        while(i<=20):
            r.append('r'+str(i));
            i+=1;        
        return r;
    
    def get_draw_columns(self):
        i=1; drs=[];
        while(i<=80):
            drs.append('dr'+str(i));
            i+=1;        
        return drs;
    
    def get_depth_columns(self):
        i=1; dps=[];
        while(i<=80):
            dps.append('dp'+str(i));
            i+=1;        
        return dps;
    
    def get_count_id(self, df=None):
        if(df is None):
            count_id = self._COUNT_ID;
        else:
            count_id = df.shape[0];
        return count_id;
    
    def get_first_id(self, df=None):
        if(df is None):
            min_id = self._MIN_ID;
        else:
            min_id = df[:1].index.item();
        return min_id;
    
    def get_last_id(self, df=None):
        if(df is None):
            max_id = self._MAX_ID;
        else:
            max_id = df[-1:].index.item();
        return max_id;
    
    def deconstruct_master_df(self):
        if(self._DF_IDs is None):
            self._DF_IDs, self._DF_R, self._DF_DR, self._DF_DP = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame();

            if (self._DF_MASTER is not None ): #or self._DF_MASTER.empty): #empty is an attribute, not a function
                print("ERROR: Missing Master Dataframe");
            else:
                self._DF_IDs = pd.DataFrame(self._DF_MASTER.index);
                self._DF_R = self._DF_MASTER.loc[:,self.get_result_columns()]
                self._DF_DR = self._DF_MASTER.loc[:,self.get_draw_columns()]
                self._DF_DP = self._DF_MASTER.loc[:,self.get_depth_columns()]

            return self._DF_IDs, self._DF_R, self._DF_DR, self._DF_DP;
    
    def df2np(self, df):
        if(isinstance(df, pd.DataFrame)):
            df = df.to_numpy();
        return df;
    
    def np2df(self, nparray):
        if(isinstance(nparray, np.ndarray)):
            nparray = pd.DataFrame(nparray);
        return nparray;
            
    def split_xy(self, df):
        x, y = pd.DataFrame(), pd.DataFrame();
        if not df.empty:
           x, y = df[:-1],df[1:];
        else:
            print("Empty dataframe")
        return x,y;
        
    def print_xy(self, x, y):
        x = self.df2np(x);
        y= self.df2np(y)
        for i in range(len(x)):
            print(x[i],y[i])
            
    def print_xy_prediction(self, y_hat, y_test):
        print("[Predicted vs Actuual]")
        
        #n_input - input rows- number of previous input timesteps t,t-1,t-2,t....n
        #t_output - Y timesteps to predict. 0 by default - predicts t+1        
    def to_supervised(self, data_df, n_input, t_output=0):
        data = self.df2np(data_df);
        X, y = list(), list()
        ix_start = 0
        # step over the entire history one time step at a time
        for i in range(len(data)):
            # define the end of the input sequence
            ix_end = ix_start + n_input
            ix_output = ix_end + t_output
            # ensure we have enough data for this instance
            if ix_output < len(data):
                X.append(data[ix_start:ix_end])
                y.append(data[ix_output])
                # move along one time step
                ix_start += 1
        return array(X), array(y);
    
    def create_supervised_package(self, masterDF, startID, nDraws, nInputs, nTests):
        #ids, X, Y, X_test, Y_test = np.ndarray([]), np.ndarray([]), np.ndarray([]), np.ndarray([]), np.ndarray([]);
        cutOff = nDraws + nInputs + nTests;
        nDFs = masterDF[startID:cutOff]
        nIDs = self.df2np(pd.DataFrame(nDFs.index))
        X_all, Y_all = self.to_supervised(nDFs,nInputs)
        
        CutSplit = -nTests; #split by 5, uses [:-5] and [-5:]
        X, Y, X_test, Y_test = self.xy_split(X_all, Y_all, CutSplit);
        IDs, IDs_test =  nIDs[nInputs:CutSplit], nIDs[CutSplit:];
        return IDs, X, Y, IDs_test, X_test, Y_test
        
        
    def xy_split(self, X_all, Y_all, CutSplit):
        x,y,x_test,y_test = X_all[:CutSplit], Y_all[:CutSplit], X_all[CutSplit:], Y_all[CutSplit:];
        return x, y, x_test, y_test;
    
    def create_new(self, newMasterFile, newMasterDF):
        self._FILE_NAME = newMasterFile;
        self.derive_folder_path();
        self.derive_file_path();
        self.derive_full_path();
        
        newMasterDF.to_pickle(self._FULL_PATH)
        print("Saved new file {0}".format(self._FULL_PATH))
        

In [6]:
import pandas as pd
import numpy as np

#from ipynb.fs.full.ifile import iFile
#from ipynb.fs.full.idata import Data

class Dataset(iFile):
    _ID = None;
    _GAME=None
    _DATASET = None;
    _DATASET_PATH = None;
    _FOLDER_TYPE = 'datasets'
    _DF_MASTER = None;
    
    def __init__(self, DatasetID='', FolderPath= 'data/', FileFormat='npz'):
        self._ID = DatasetID;
        super().__init__(DatasetID, FolderPath, FileFormat)
        self.decipher_file_name();
        self.derive_folder_path(self._INFO); #you don't need this for model
        self.derive_file_path();
        self.derive_full_path();
    
    def decipher_file_name(self, delimiter="_"):
        #if .format is included, remove this first
        FN = self._FILE_NAME
        FN_PARTS = FN.split(delimiter);
        self._INFO['GAME'], self._GAME = FN_PARTS[0], FN_PARTS[0]
        self._INFO['xnINPUTS'] = FN_PARTS[1]
        self._INFO['nINPUTS'] = self.derive_xninputs(self._INFO['xnINPUTS'])
        self._INFO['xnDRAWS'] = FN_PARTS[2]
        self._INFO['nDRAWS'] = self.derive_xndraws(self._INFO['xnDRAWS'])
        self._INFO['DATA_TYPE'] = FN_PARTS[3] 
        #check if file format
        #self.set_file_format(self._FILE_NAME); #can't use this as we use decimal

        return self._FILE_NAME_DECIPHERS;
    
    #25000=>25k and 25k=>25000
    def derive_xndraws(self, xndraws):
        nDraw = 0; notation=''
        if (type(xndraws) == str):
            notation=xndraws[len(xndraws)-1];            
            if(type(notation) == int): multiple = 1;
            elif (notation=='k'): multiple = 1000;
            elif (notation=='m'): multiple = 1000000;
            new_st = float(xndraws.replace(notation,''))# change to float 2.5 first
            nDraw = int(new_st*multiple) #then covert back to int from float
            print("xndraw{0}, new_st{1}, multiple{2}, notation{3}".format(xndraws, new_st, multiple, notation))
        elif (type(xndraws) == int):
            if(xndraws > 1000000) :
                notation = 'm';
                nDraw = str(xndraws//1000000)+notation;
            elif (xndraws > 1000):
                notation = 'k';
                nDraw = str(xndraws//1000)+notation;
            else:
                nDraw = str(xndraws//1);
        return nDraw;
    
    #x15=>15 and 15=>x15
    def derive_xninputs(self, xnInputs):
        nInputs = 0;
        if (type(xnInputs) == str):
            #remove first character
            nInputs = int(xnInputs[1:]);
        elif (type(xnInputs) == int):
            nInputs = 'x'+str(xnInputs);        
        return nInputs;
    

    def set_file_format(self, FileName):
        #split name and see if file format is included
        Fs=FileName.split(".");
        try:
          self._FILE_FORMAT=Fs[1];
        except IndexError:
          print("File format not included in FIle Name")
        return self._FILE_FORMAT;

    def derive_file_path(self):
        #derive file name from decipher  
        FilePath = self._FOLDER_TYPE + '/';
        FilePath += self._FILE_NAME;
        FilePath += "." + self._FILE_FORMAT;
        self._FILE_PATH = FilePath;
        print("[iDataset:derive_file_path] {0}".format(self._FILE_PATH))
        
    def load(self):
        self._DATASET = np.load(self._FULL_PATH)
        print("Loading dataset {0}".format(self._FULL_PATH))
        
    def load_master_df(self):
        if(self._DF_MASTER is None):
            masterPkl = self._GAME +"_master";
            masterDF = Data(masterPkl);
            self._DF_MASTER = masterDF._DF_MASTER;
            print("[iDataset:load_master_df] Loaded master DF")
            
    def derive_full_path_by_id(self, newDatasetID):
        self._FILE_NAME = newDatasetID; #this is the onlyl thing you need to change
        self.decipher_file_name();
        self.derive_folder_path(self._INFO); #you don't need this for model
        self.derive_file_path();
        self.derive_full_path();
        
    def create_new_by_id(self, DF_Master, newDatasetID, nTests=15):
        self.derive_full_path_by_id(newDatasetID);
        self.ensure_dirs(); #ensure Full Path exists
        #now use file info details self._INFO
        
        masterPkl = self._INFO['GAME'] +"_master";
        masterData = Data(masterPkl);
        ids, x, y, ids_test, x_test, y_test=masterData.create_supervised_package(DF_Master, 0, self._INFO['nDRAWS'], self._INFO['nINPUTS'], nTests);
        
        np.savez(self._FULL_PATH, IDs=ids, X=x, Y=y, IDs_test=ids_test, X_test=x_test, Y_test=y_test);
        
        print("Saved new datast file {0}".format(self._FULL_PATH))
        
    def derive_new_dataset_id(self, Game, nDraws, nInputs, DataType='dr'):
        newDatasetID = Game + "_";
        newDatasetID += self.derive_xninputs(nInputs) + "_";
        newDatasetID += self.derive_xndraws(nDraws) + "_";
        newDatasetID += DataType;
        return newDatasetID;
        
    def create_new(self, DF_Master, startID, nDraws, nInputs, nTests=15, DataType='dr'):        
        newDatasetID = self.derive_new_dataset_id(self._INFO['GAME'], nDraws, nInputs, DataType);
        
        self.create_new_by_id(DF_Master, newDatasetID, nTests)

In [7]:
from ipynb.fs.full.imodel import Model
import keras as kr
from keras.models import Sequential
from keras.layers import LSTM, Dense

class KER_Model(Model):
    _MODEL_PATH = None;
    
    def __init__(self, ModelID, Dataset):
        super().__init__(ModelID, Dataset)
        print("[KER_Model:__init__]")
 
    def load(self):
        print("[KER_Model:load]: Loading {0}".format(self._FULL_PATH))
        pass;
    
    def save(self):
        print("[KER_Model:save]: Saving {0}".format(self._FULL_PATH))
    
    def load_best_version(self):
        pass;
    
    def load_latest_version(self):
        pass;
        
    
    

In [8]:
from ipynb.fs.full.imodel import Model
import tensorflow as tfl

class TFL_Model(Model):
    _MODEL_PATH = None;
    
    def __init__(self, ModelID, Dataset):
        super().__init__(ModelID, Dataset)
 
    def load(self):
        print("Loading {0}".format(self._FULL_PATH))
        pass;
    
    def save(self):
        print("Saving {0}".format(self._FULL_PATH))
    
    def load_best_version(self):
        pass;
    
    def load_latest_version(self):
        pass;
        
    
    

In [9]:
from ipynb.fs.full.ker_model import KER_Model;
from ipynb.fs.full.tfl_model import TFL_Model;
from ipynb.fs.full.idataset import Dataset

class AI:
    _M =None;
    _D =None;
    _MASTER=None;
    _GAME = None;
    _API = None;
    
    def __init__(self, ModelID, DatasetID):
        self.load_dataset(DatasetID)
        self._D.ensure_dirs();
        
        self.load_model(ModelID, self._D); #you need x5/25k from datasetfolder
        self._M.ensure_dirs();
        
        print("[AI:__init__]")
        pass;

    
    def setup_model(self, ModelName):
        pass;
    
    def load_model(self, ModelID, Dataset):
        info = ModelID.split(".");
        self._GAME = info[0];
        self._API = info[1]
        
        print("Game: {0} API {1}".format(self._GAME, self._API))
        
        if (self._API == 'ker'):
            self._M = KER_Model(ModelID, Dataset);
        elif(self._API == 'tfl'):
            self._M = TFL_Model(ModelID, Dataset);
            
        self._M.load();
        print("Loaded Model {0}".format(self._M._FULL_PATH))
    
    def load_dataset(self, DatasetName):
        self._D = Dataset(DatasetName);
        self._D.load();
        #Dataset is stored in self._D._DATASET[IDs,X,Y,IDs_test,X_test,Y_test]
        print("Loaded Dataset {0}".format(self._D._FULL_PATH))
    
    def save_model(self, Model):
        pass;
    
    def get_model_summary(self, Model):
        pass;
    
    def train_model(self, Model, TrainingDataset):
        pass;
    
    def predict_model(self, Model, TestDataset):
        pass;
    
    

xndraw2.528m, new_st2.528, multiple1000000, notationm
[iFile:__init__]
xndraw2.528m, new_st2.528, multiple1000000, notationm
[iFile:derive_folder_path] data/keno/x15/2.528m/
[iDataset:derive_file_path] datasets/keno_x15_2.528m_dr.npz
[ifile:derive_full_path] ../data/keno/x15/2.528m/datasets/keno_x15_2.528m_dr.npz
Loading dataset ../data/keno/x15/2.528m/datasets/keno_x15_2.528m_dr.npz


In [ ]:
ai = AI('ker.1.1','keno_x15_100k_dr')